In [6]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import preprocessing
import importlib

importlib.reload(preprocessing)

<module 'preprocessing' from 'C:\\Users\\dbsrh\\OneDrive\\Desktop\\Pytorch\\kaggle_digit_recognizer\\preprocessing.py'>

In [3]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=3)
        self.conv2 = torch.nn.Conv2d(10, 15, kernel_size=3)
        self.fc1 = torch.nn.Linear(375, 50)
        self.fc2 = torch.nn.Linear(50, 10)
        
    def forward(self, x):
        # 28 X 28 --> 26 X 26 --> 13 X 13
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        # 13 X 13 --> 11 X 11 --> 5 X 5
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        # 5 X 5 X 15
        x = x.view(-1, 375)
        x = self.fc1(x)
        x = F.dropout(x, p=0.3, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [13]:
# Define CNN model
model_ft = Model()

# Load data
valid_fraction = 0.1
train_data = preprocessing.DigitDataset('./digit-recognizer/train.csv')
boundary = int(len(train_data) * (1 - 0.1))

train_data_loader = DataLoader(
    train_data[:boundary],
    batch_size = 32,
    shuffle = True,
    num_workers = 3
)

valid_data_loader = DataLoader(
    train_data[:boundary],
    batch_size = 32,
    shuffle = True,
    num_workers = 3
)

datasets = {'train' : train_data_loader, 'valid' : valid_data_loader}
datasets_size = {'train' : len(train_data_loader.dataset), 'valid' : len(valid_data_loader.dataset)}

42000
